The dataset is designed for estimating the accurate count of people present in a room through the utilization of various non-intrusive environmental sensors. These sensors include devices that measure factors such as temperature, light intensity, sound levels, carbon dioxide (CO2) concentration, and passive infrared (PIR) motion detection.

By collecting data from these sensors, it becomes possible to discern patterns and correlations between the sensor readings and the actual number of occupants in the room. For instance, the dataset could reveal that when the temperature rises, the number of occupants tends to increase, or that higher sound levels are associated with larger groups of people.

This type of dataset has significant potential applications, such as in building automation systems, energy efficiency optimization, and occupancy-based services. It enables researchers, engineers, and data scientists to develop algorithms and models that can accurately estimate the occupancy of a space based on the readings from these sensors. This information could be crucial for managing resources, improving comfort, and enhancing safety in various environments, from offices and commercial spaces to residential buildings.

## Additional Information

- Date: YYYY/MM/DD
- Time: HH:MM:SS
- Temperature: In degree Celsius
- Light: In Lux
- Sound: In Volts (amplifier output read by ADC)
- CO2: In PPM
- CO2 Slope: Slope of CO2 values taken in a sliding window
- PIR: Binary value conveying motion detection
- Room_Occupancy_Count: Ground Truth

In [1]:
import pandas as pd
import numpy as np

In [2]:
Room_Occ = pd.read_csv('/content/drive/Othercomputers/My Laptop/ML_Bash/Room Occupancy Estimation/Occupancy_Estimation.csv')

In [3]:
Room_Occ

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124,2018/01/11,08:58:07,25.06,25.13,24.69,25.31,6,7,33,22,0.09,0.04,0.06,0.08,345,0.000000,0,0,0
10125,2018/01/11,08:58:37,25.06,25.06,24.69,25.25,6,7,34,22,0.07,0.05,0.05,0.08,345,0.000000,0,0,0
10126,2018/01/11,08:59:08,25.13,25.06,24.69,25.25,6,7,34,22,0.11,0.05,0.06,0.08,345,0.000000,0,0,0
10127,2018/01/11,08:59:39,25.13,25.06,24.69,25.25,6,7,34,22,0.08,0.08,0.10,0.08,345,0.000000,0,0,0
